In [2]:
conda install -c conda-forge wordcloud

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\havu1\anaconda3

  added / updated specs:
    - wordcloud


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.10.3               |   py37h03978a9_2         3.1 MB  conda-forge
    msgpack-python-1.0.2       |   py37h8c56517_1          80 KB  conda-forge
    openssl-1.1.1l             |       h8ffe710_0         5.7 MB  conda-forge
    wordcloud-1.8.1            |   py37hcc03f2d_1         194 KB  conda-forge
    wrapt-1.12.1               |   py37hcc03f2d_3          46 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         9.1 MB

The following NEW packages will be INSTALLED:

  msgpack-python     conda-forge/win-64::msgpack-python-1.0.2-py37h8c56517_1
  wordcloud          conda-forge/win-64::wordcloud-1.8.1-


The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/win-64::anaconda==custom=py37_1
  - defaults/win-64::astroid==2.6.2=py37haa95532_0
  - defaults/noarch::dask==2021.7.0=pyhd3eb1b0_0
  - defaults/win-64::distributed==2021.7.0=py37haa95532_0
  - defaults/win-64::keras==2.2.4=0
  - defaults/win-64::pylint==2.9.3=py37haa95532_1
  - defaults/noarch::pyls-black==0.4.6=hd3eb1b0_0
  - defaults/noarch::pyls-spyder==0.3.2=pyhd3eb1b0_0
  - defaults/noarch::python-language-server==0.36.2=pyhd3eb1b0_0
  - defaults/win-64::spyder==3.3.6=py37_0
  - defaults/win-64::tensorflow==1.14.0=gpu_py37h5512b17_0
  - defaults/win-64::tensorflow-base==1.14.0=gpu_py37h55fc52a_0
  - defaults/noarch::tensorflow-estimator==1.14.0=py_0
  - defaults/win-64::tensorflow-gpu==1.14.0=h0d30ee6_0
  - defaults/noarch::tensorflow-probability==0.7=py_2
  - defaults/win-64::_anaconda_depends==2020.07=py37_0


In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords") 
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

from wordcloud import WordCloud, STOPWORDS

import os

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\havu1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\havu1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\havu1\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


In [2]:
data = pd.read_csv(r"./Downloads/Sheet_1.csv",encoding= "latin1" )
data.drop(["Unnamed: 3","Unnamed: 4","Unnamed: 5",
           "Unnamed: 6","Unnamed: 7",], axis = 1, inplace =True)
data = pd.concat([data["class"],data["response_text"]], axis = 1)

data.dropna(axis=0, inplace =True)
data.head(10)

,class,response_text
0,not_flagged,I try and avoid this sort of conflict
1,flagged,Had a friend open up to me about his mental ad...
2,flagged,I saved a girl from suicide once. She was goin...
3,not_flagged,i cant think of one really...i think i may hav...
4,not_flagged,Only really one friend who doesn't fit into th...
5,not_flagged,a couple of years ago my friends was going to ...
6,flagged,Roommate when he was going through death and l...
7,flagged,i've had a couple of friends (you could say mo...
8,not_flagged,Listened to someone talk about relationship tr...
9,flagged,I will always listen. I comforted my sister wh...


In [3]:
data["class"] = [1 if each == "flagged" else 0 for each in data["class"]]
data.head()

,class,response_text
0,0,I try and avoid this sort of conflict
1,1,Had a friend open up to me about his mental ad...
2,1,I saved a girl from suicide once. She was goin...
3,0,i cant think of one really...i think i may hav...
4,0,Only really one friend who doesn't fit into th...


In [4]:
data.response_text[16]

'I have helped advise friends who have faced circumstances similar to mine'

In [5]:
first_text = data.response_text[16]
text = re.sub("[^a-zA-Z]"," ",first_text)
text = text.lower() 
print(text)

i have helped advise friends who have faced circumstances similar to mine


In [8]:
text = nltk.word_tokenize(text)
text = [ word for word in text if not word in set(stopwords.words("english"))]
print(text)

['helped', 'advise', 'friends', 'faced', 'circumstances', 'similar', 'mine']


In [11]:
lemmatizer = WordNetLemmatizer()
text = [(lemmatizer.lemmatize(lemmatizer.lemmatize(lemmatizer.lemmatize(word, "n"),pos = "v"),pos="a")) for word in text]
print(text)

['help', 'advise', 'friend', 'face', 'circumstance', 'similar', 'mine']


In [12]:
description_list = []
for description in data.response_text:
       
    description = re.sub("[^a-zA-Z]"," ",description)
    description = description.lower() 
    
    description = nltk.word_tokenize(description)
    description = [ word for word in description if not word in set(stopwords.words("english"))]
    
    lemmatizer = WordNetLemmatizer()
    description = (lemmatizer.lemmatize(lemmatizer.lemmatize(lemmatizer.lemmatize(word, "n"),pos = "v"),pos="a") for word in description)
    
    description = " ".join(description)
    description_list.append(description)

In [13]:
description_list[16]

'help advise friend face circumstance similar mine'

In [14]:
max_features = 100
count_vectorizer = CountVectorizer(max_features=max_features)
sparce_matrix = count_vectorizer.fit_transform(description_list).toarray()
print("Top {} Most Used Words: {}".format(max_features,count_vectorizer.get_feature_names()))

Top 100 Most Used Words: ['addiction', 'advice', 'alone', 'always', 'anxiety', 'anything', 'back', 'best', 'bring', 'call', 'care', 'come', 'comfort', 'could', 'deal', 'depression', 'describe', 'dont', 'end', 'even', 'everything', 'experience', 'face', 'feel', 'find', 'friend', 'get', 'gf', 'girl', 'girlfriend', 'give', 'go', 'good', 'grade', 'happen', 'help', 'helpful', 'issue', 'kid', 'kill', 'know', 'last', 'let', 'life', 'like', 'listen', 'little', 'look', 'lot', 'make', 'many', 'may', 'much', 'need', 'never', 'night', 'offer', 'often', 'one', 'open', 'others', 'people', 'person', 'personal', 'pretty', 'problem', 'really', 'relationship', 'say', 'school', 'see', 'self', 'severe', 'share', 'shit', 'similar', 'simply', 'situation', 'someone', 'sometimes', 'start', 'struggle', 'stuff', 'suicide', 'support', 'talk', 'tell', 'think', 'though', 'time', 'trouble', 'try', 'use', 'want', 'way', 'week', 'well', 'work', 'would', 'year']


In [15]:
y = data.iloc[:,0].values
x = sparce_matrix

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 42)